In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO
from tqdm import tqdm

In [2]:
# Define the Butterworth low-pass filter function
def apply_dft_butterworth_filter(image, d0=100, n=2):
    # Perform DFT on each color channel
    channels = cv2.split(image)
    filtered_channels = []
    
    for channel in channels:
        # Perform DFT
        dft = cv2.dft(np.float32(channel), flags=cv2.DFT_COMPLEX_OUTPUT)
        dft_shift = np.fft.fftshift(dft)
        
        # Create Butterworth low-pass filter mask
        rows, cols = channel.shape
        crow, ccol = rows // 2, cols // 2
        mask = np.zeros((rows, cols, 2), np.float32)
        for u in range(rows):
            for v in range(cols):
                dist = np.sqrt((u - crow)**2 + (v - ccol)**2)
                mask[u, v] = 1 / (1 + (dist / d0)**(2 * n))
        
        # Apply mask and inverse DFT
        fshift = dft_shift * mask
        f_ishift = np.fft.ifftshift(fshift)
        img_back = cv2.idft(f_ishift)
        img_back = cv2.magnitude(img_back[:,:,0], img_back[:,:,1])
        
        # Normalize the result
        cv2.normalize(img_back, img_back, 0, 255, cv2.NORM_MINMAX)
        filtered_channels.append(np.uint8(img_back))
    
    # Merge the filtered channels back
    filtered_image = cv2.merge(filtered_channels)
    
    return filtered_image

In [ ]:
# Define the paths
input_folder = 'datasets/Liver_Ultrasound_Detection_data/liver-ultrasound-detection-unzip/images/subsets/subset_1'
output_folder = 'datasets/Liver_Ultrasound_Detection_data/liver-ultrasound-detection-unzip/images/subsets/Prepro'

# Create output directory if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get a list of all image files in the input folder
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]  # Add other extensions if needed

# Process each image in the input folder
for filename in tqdm(image_files, desc="Processing images"):
    image_path = os.path.join(input_folder, filename)
    img = cv2.imread(image_path)

    # Apply Butterworth filter to remove Moiré patterns
    filtered_img = apply_dft_butterworth_filter(img, d0=125, n=2)  # Adjust d0 and n as needed

    # Convert the filtered image to grayscale
    grayimg = cv2.cvtColor(filtered_img, cv2.COLOR_BGR2GRAY)

    # Normalize the image
    arr_normalized = cv2.normalize(grayimg, None, 0, 255, cv2.NORM_MINMAX)
    arr_normalized = arr_normalized.astype(np.uint8)

    # Save preprocessed image to output folder
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, arr_normalized)

Processing images:  89%|████████▉ | 460/516 [5:56:10<48:18, 51.76s/it]  